In [1]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [2]:
from pyspark.sql.functions import udf
import pyspark.sql.functions as fct

from pyspark.sql import Row
from geopy.distance import distance as geo_dist

import pandas as pd 

from ipywidgets import interact, interactive, fixed, interact_manual, widgets
import numpy as np
from pyspark.sql.types import *
from pyspark.sql.functions import when, col

import requests
import time

In [3]:
import getpass
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('final_proj-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '4g')
conf.set('spark.executor.instances', '10')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=final_proj-granoux>

In [4]:
spark = SparkSession(sc)

Load Data 

In [5]:
df = spark.read.csv('/datasets/project/istdaten/*/*/*', sep=';', header=True)

Rename columns: 

In [6]:
columns = 'TripDate string, TripId string, OperatorId string, OperatorAbbrv string, OperatorName string, ProductId string, LineId string, LineType string, UmlaufId string, TransportType string, AdditionalTrip boolean, FailedTrip boolean, BPUIC string, StopName string, ArrivalTimeScheduled string, ArrivalTimeActual string, ArrivalTimeActualStatus string,     DepartureTimeScheduled string, DepartureTimeActual string, DepartureTimeActualStatus string, SkipStation boolean'
columns = list(map(lambda x: x.split()[0],columns.split(',')))

for old, new in zip(df.columns, columns):
    #print(old, new)
    df = df.withColumnRenamed(old, new)

In [7]:
df.printSchema()

root
 |-- TripDate: string (nullable = true)
 |-- TripId: string (nullable = true)
 |-- OperatorId: string (nullable = true)
 |-- OperatorAbbrv: string (nullable = true)
 |-- OperatorName: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- LineId: string (nullable = true)
 |-- LineType: string (nullable = true)
 |-- UmlaufId: string (nullable = true)
 |-- TransportType: string (nullable = true)
 |-- AdditionalTrip: string (nullable = true)
 |-- FailedTrip: string (nullable = true)
 |-- BPUIC: string (nullable = true)
 |-- StopName: string (nullable = true)
 |-- ArrivalTimeScheduled: string (nullable = true)
 |-- ArrivalTimeActual: string (nullable = true)
 |-- ArrivalTimeActualStatus: string (nullable = true)
 |-- DepartureTimeScheduled: string (nullable = true)
 |-- DepartureTimeActual: string (nullable = true)
 |-- DepartureTimeActualStatus: string (nullable = true)
 |-- SkipStation: string (nullable = true)



In [8]:
df_tmp = df.select(fct.split(df.TripId, ':')[2].alias('Line_ID'),
              fct.split(df.TripId, ':')[3].alias('Line_ID_spec'),
              'LineType', 'ProductId','LineType', 'TripDate', 
                   'ArrivalTimeScheduled','DepartureTimeScheduled', 'StopName')

### Metadata

We first start by read metadata in order to select stop station within 10 km from Zürich 

In [9]:
df_meta = spark.read.csv('/datasets/project/metadata')

In [10]:
#Here we can see that we have some duplicated stop name 
#df_meta.filter(df_meta._c0.contains('Zürich')).head(30)

In [11]:
df_meta.printSchema()

root
 |-- _c0: string (nullable = true)



In [12]:
test = df_meta.filter(df_meta['_c0'].rlike("Lausanne")).collect()
len(test)

211

In [13]:
df_meta = df_meta.select(fct.split(df_meta['_c0'], '  ')[1].alias('Long'), 
                         fct.split(fct.split(df_meta['_c0'], '  ')[2], ' ')[0].alias('Lat'), 
                         fct.split(df_meta['_c0'], '% ')[1].alias('StopName_Meta') )

In [14]:
df_meta.show()

+---------+---------+-------------------+
|     Long|      Lat|      StopName_Meta|
+---------+---------+-------------------+
|26.074412|44.446770|          Bucuresti|
| 1.811446|50.901549|             Calais|
| 1.075329|51.284212|         Canterbury|
|-3.543547|50.729172|             Exeter|
| 9.733756|46.922368|            Fideris|
| 8.571251|50.051219|Frankfurt Flughafen|
|18.643803|54.355520|             Gdansk|
| 7.389462|47.191804|           Grenchen|
|29.019602|40.996348|           Istanbul|
| 9.873959|48.577852|  Amstetten (Württ)|
| 4.786044|43.921937|            Avignon|
| 2.140369|41.378914|          Barcelona|
| 7.589551|47.547405|              Basel|
| 7.395229|46.937482|       Bern Bümpliz|
|-1.899480|52.483627|         Birmingham|
| 6.838953|46.949588|          Boudry TN|
|17.106466|48.158910|         Bratislava|
| 4.335694|50.835376|          Bruxelles|
|-2.979650|53.404289|          Liverpool|
| 8.500049|47.114619|         Lothenbach|
+---------+---------+-------------

In [15]:
print(len(df_meta.filter(df_meta['StopName_Meta'].rlike("Lausanne")).collect()))

211


In [16]:
#Again we can see that we have many occurance of Zurich with different coordinate
#df_meta.filter(df_meta.StopName_Meta == 'Zürich').show()

In [17]:
df_meta = df_meta.withColumn("Long", df_meta["Long"].cast(FloatType()))
df_meta = df_meta.withColumn("Lat", df_meta["Lat"].cast(FloatType()))

In [18]:
df_meta.printSchema()

root
 |-- Long: float (nullable = true)
 |-- Lat: float (nullable = true)
 |-- StopName_Meta: string (nullable = true)



We can see that there is many duplicate name with different coordinate. 
For example we find many time Lausanne, after investigatin we understand that all the subway station where simply Lausanne. We decide to fill that problem using another dataset in order to merge them. 

We decide to merge the two dataset using coordinate, in order to do this we round coordinate to match them. A round at 3 decimal change the precission by max 135m. For example Google Maps use 6 decimal

First we only keep point in/near switzerland we decide to do this by draw a square arount the country and keep point inside.  Here we find the extreme points of switzerland: 
https://fr.wikipedia.org/wiki/Liste_de_points_extr%C3%AAmes_de_la_Suisse

In [19]:
df_meta.count()

25935

In [20]:
df_meta = df_meta.filter(df_meta.Lat.between(45.490404, 47.485074))
df_meta = df_meta.filter(df_meta.Long.between(5.572263, 10.2931))                       

In [21]:
df_meta.count()

22723

Then we see the minimum precision we have in our dataset in order to round all coordinate to this precision

In [22]:
slen = udf(lambda s: len(str(s).split('.')[1]), IntegerType())

In [23]:
df_meta = df_meta.withColumn("lat_len", slen(df_meta.Lat))
df_meta = df_meta.withColumn("lon_len", slen(df_meta.Long))
#df_meta = df_meta.withColumn("precision", min(df_meta.lat_len, df_meta.lon_len))
print(df_meta.agg({"lat_len": "min"}).collect())
print(df_meta.agg({"lon_len": "min"}).collect())

[Row(min(lat_len)=6)]
[Row(min(lon_len)=6)]


So we have a precision of 6 digit which is sufficient for our work. See why df_meta.show(5) not always display the same number of digit 

In [24]:
df_meta.show(5)

+--------+---------+-------------+-------+-------+
|    Long|      Lat|StopName_Meta|lat_len|lon_len|
+--------+---------+-------------+-------+-------+
|9.733756|46.922367|      Fideris|     15|     15|
|7.389462|47.191803|     Grenchen|     15|     15|
|7.395229| 46.93748| Bern Bümpliz|     14|     15|
|6.838953| 46.94959|    Boudry TN|     15|     15|
|8.500049| 47.11462|   Lothenbach|     15|     15|
+--------+---------+-------------+-------+-------+
only showing top 5 rows



In [25]:
df_meta = df_meta.select('Long', 'Lat', 'StopName_Meta')

In [26]:
round_6 = udf(lambda s: round(s, 6), DoubleType())

In [27]:
df_meta = df_meta.withColumn("Round_Long", round_6(df_meta.Long))
df_meta = df_meta.withColumn("Round_Lat", round_6(df_meta.Lat))

In [28]:
df_meta.show(5)

+--------+---------+-------------+----------+---------+
|    Long|      Lat|StopName_Meta|Round_Long|Round_Lat|
+--------+---------+-------------+----------+---------+
|9.733756|46.922367|      Fideris|  9.733756|46.922367|
|7.389462|47.191803|     Grenchen|  7.389462|47.191803|
|7.395229| 46.93748| Bern Bümpliz|  7.395229|46.937481|
|6.838953| 46.94959|    Boudry TN|  6.838953|46.949589|
|8.500049| 47.11462|   Lothenbach|  8.500049| 47.11462|
+--------+---------+-------------+----------+---------+
only showing top 5 rows



In [29]:
print(df_meta.distinct().count())
print(df_meta.select('Round_Lat', 'Round_Long').distinct().count())

22696
22671


### Use another dataset to fil missong name

In [30]:
with open('stops.txt', 'r') as file: 
    one_splitted = file.readline().strip().split(",")
    file_lines = [line.strip().split('"') for line in file.readlines()]
    
stop_names = [x[3] for x in file_lines]
Lat = [float(x[5]) for x in file_lines]
Long = [float(x[7]) for x in file_lines]

df_stop = pd.DataFrame({
        "StopName": stop_names, 
        "Lat_stop": Lat, 
        "Long_stop": Long,   
    })
df_stop.head()

,Lat_stop,Long_stop,StopName
0,45.989901,8.345062,"Anzola, chiesa"
1,46.167251,8.345807,Altoggio
2,46.060122,8.113620,Antronapiana
3,45.989870,8.345717,Anzola
4,46.261498,8.319253,Baceno


In [31]:
mySchema = StructType([ StructField("Lat_stop", DoubleType(), True)\
                        ,StructField("Long_stop", DoubleType(), True)\
                        ,StructField("StopName", StringType(), True) ])
df_stop = spark.createDataFrame(df_stop, mySchema)
df_stop.show()

+----------------+----------------+--------------------+
|        Lat_stop|       Long_stop|            StopName|
+----------------+----------------+--------------------+
|45.9899010293845|8.34506152974108|      Anzola, chiesa|
|46.1672513851495|  8.345807131427|            Altoggio|
| 46.060121674738|8.11361957990831|        Antronapiana|
|45.9898698225697|8.34571729989858|              Anzola|
|46.2614983591677|8.31925293162473|              Baceno|
|46.0790618438814|8.29927439970313|Beura Cardezza, c...|
|46.1222963432243|8.21077237789936|Bognanco, T. Vill...|
|46.0656504576122|8.26113193273411|           Boschetto|
|46.2978807772998| 8.3626325767009|            Cadarese|
|46.1340194356792|8.28619492916453|               Caddo|
|46.0916476333918|8.28041876188684|              Calice|
|45.9695691829797|8.04585965801774|            Campioli|
|46.4091810825782| 8.4117524564434|    Cascate del Toce|
|46.0205875326422| 8.2148866619012|         Castiglione|
|45.9710364221151|8.06992552448

In [32]:
df_stop = df_stop.withColumn("lat_len", slen(df_stop.Lat_stop))
df_stop = df_stop.withColumn("lon_len", slen(df_stop.Long_stop))
#df_meta = df_meta.withColumn("precision", min(df_meta.lat_len, df_meta.lon_len))
print(df_stop.agg({"lat_len": "min"}).collect())
print(df_stop.agg({"lon_len": "min"}).collect())

df_stop.orderBy('lon_len').show(2)

[Row(min(lat_len)=1)]
[Row(min(lon_len)=1)]
+----------------+------------+--------------------+-------+-------+
|        Lat_stop|   Long_stop|            StopName|lat_len|lon_len|
+----------------+------------+--------------------+-------+-------+
|             0.0|         0.0|     Isola Superiore|      1|      1|
|47.3611471419894|7.3110197892|Develier, St-Chri...|     13|     10|
+----------------+------------+--------------------+-------+-------+
only showing top 2 rows



In [33]:
print(df_stop.filter(df_stop['StopName'].rlike("Isola Superiore")).collect())

[Row(Lat_stop=0.0, Long_stop=0.0, StopName='Isola Superiore', lat_len=1, lon_len=1)]


Here we can see that this can from an error in the dataset, we use google maps to find the good coordinate of Isola Superiore which is: Isola Superiore: 45.901230 - 8.520450

In [34]:
df_stop = df_stop.withColumn("Lat_stop", \
              when(df_stop["StopName"] == 'Isola Superiore', 45.901230).otherwise(df_stop["Lat_stop"]))
df_stop = df_stop.withColumn("Long_stop", \
              when(df_stop["StopName"] == 'Isola Superiore', 8.520450).otherwise(df_stop["Long_stop"]))

Now we again round all the coordinate by 6 in order to merge both of the dataframe. 

In [35]:
df_stop = df_stop.withColumn("Round_Long", round_6(df_stop.Long_stop))
df_stop = df_stop.withColumn("Round_Lat", round_6(df_stop.Lat_stop))

### Merge Dataframe

In [36]:
Df_meta = df_meta.join(df_stop, on = ['Round_Lat', 'Round_Long'], how='outer') 

In [37]:
print(df_meta.filter(df_meta['StopName_Meta'].like("Lausanne")).count())
print(Df_meta.filter(Df_meta['StopName_Meta'].like("Lausanne") & Df_meta['StopName'].isNull()).count())

188
185


We can see that for the example of Lausanne we just recover 2 name over about a hundred. 
After investigation we find the coordidate for particular station in both dataset: 
<br/>
<br/>Lausanne Malley: 46.524212 - 6.603306 -- 46.524211 - 6.603309
<br/>Lausanne Bourdonette: 46.523466 - 6.589805 -- 46.523465 - 6.589807
<br/>Lausanne Provence: 46.523384 - 6.608102 -- 46.523382 - 6.608106

We can see that each time our merge fail for 1 digit

We try again with a round at 5 digits whith is still a very good precision

In [38]:
round_5 = udf(lambda s: round(s, 5), DoubleType())

In [39]:
df_meta = df_meta.withColumn("Round_Long", round_5(df_meta.Long))
df_meta = df_meta.withColumn("Round_Lat", round_5(df_meta.Lat))

In [40]:
df_stop = df_stop.withColumn("Round_Long", round_5(df_stop.Long_stop))
df_stop = df_stop.withColumn("Round_Lat", round_5(df_stop.Lat_stop))

In [41]:
Df_meta = df_meta.join(df_stop, on = ['Round_Lat', 'Round_Long'], how='outer') 

In [42]:
print(df_meta.filter(df_meta['StopName_Meta'].like("Lausanne")).count())
print(Df_meta.filter(Df_meta['StopName_Meta'].like("Lausanne") & Df_meta['StopName'].isNull()).count())

188
65


We now achieved a satisfactory result

In [43]:
Df_meta.printSchema()

root
 |-- Round_Lat: double (nullable = true)
 |-- Round_Long: double (nullable = true)
 |-- Long: float (nullable = true)
 |-- Lat: float (nullable = true)
 |-- StopName_Meta: string (nullable = true)
 |-- Lat_stop: double (nullable = true)
 |-- Long_stop: double (nullable = true)
 |-- StopName: string (nullable = true)
 |-- lat_len: integer (nullable = true)
 |-- lon_len: integer (nullable = true)



In [44]:
Df_meta = Df_meta.select('Long', 'Lat', 'StopName_Meta', 'StopName')
Df_meta.show()

+--------+---------+----------------+--------------------+
|    Long|      Lat|   StopName_Meta|            StopName|
+--------+---------+----------------+--------------------+
|    null|     null|            null|Macugnaga, Pestarena|
|    null|     null|            null| Lugano, Via Ginevra|
|    null|     null|            null|      Gandria, Paese|
|    null|     null|            null|               Gozzi|
|8.943882|46.034714|        Cureglia|   Cureglia, Rotonda|
|    null|     null|            null|        Bogno, Paese|
|6.090986| 46.15237|           Perly|                null|
|6.044045|46.161507|        Laconnex|Laconnex, Chemin ...|
|8.912559|46.179436|         Agarone|                null|
|8.699336| 46.18245|        Cresmino|      Cresmino, Case|
|6.246757|46.183704|       Annemasse|Annemasse, Généra...|
|7.393176| 46.19771|Les Mayens-de-S.|Les Mayens-de-S.,...|
|6.167676| 46.20001|          Genève|  Genève, Amandolier|
|6.157857|46.203766|          Genève|                nul

In [45]:
Df_meta = Df_meta.na.drop(subset=["Long", 'Lat'])

In [46]:
Df_meta = Df_meta.withColumn("StopName_Meta", \
              when(Df_meta["StopName"].isNotNull(), Df_meta["StopName"]).otherwise(Df_meta["StopName_Meta"]))

In [47]:
Df_meta = Df_meta.select('StopName_Meta', 'Lat', 'Long')

### Request Part

In [48]:
def get_lat_long(name): 
    tmp = Df_meta.select('Lat', 'Long').filter(Df_meta['StopName_Meta'].like(name)).collect()
    if(len(tmp) == 0): 
        assert "Problement with the location {}".format(name)
    #print(tmp)
    tmp = tmp[0]
    print(tmp)
    lat = str(tmp).split('=')[1].split(',')[0]
    long = str(tmp).split('=')[2].split(')')[0]
    return lat, long

In [49]:
get_lat_long('Lausanne')

Row(Lat=46.53168869018555, Long=6.628243923187256)


('46.53168869018555', '6.628243923187256')

In [50]:
def request(fromPlace, toPlace, departure,Hours, AM_PM,Minutes, Months, Days):
    lat_from, long_from = get_lat_long(fromPlace)
    lat_to, long_to = get_lat_long(toPlace)
    url = 'http://10.90.38.21:8829/otp/routers/default/plan?fromPlace=stop+'
    url += '+'.join(fromPlace.split()) +  '+%3A%3A' + str(lat_from) + '%2C' + str(long_from)
    url += '&toPlace=stop+' +  '+'.join(toPlace.split()) +  '+%3A%3A' + str(lat_to) + '%2C' + str(long_to)
    url += '&time={}%3A{}{}&date={}-{}-2018&mode=TRANSIT%2CWALK&maxWalkDistance=804.672&arriveBy=true&wheelchair=false&locale=en'.format(Hours, Minutes,AM_PM, Months, Days)
    print(url)
    r = requests.get(url)
    print(r)
    #print(r.json())
    read_json(r.json())
    #return r.json()

In [58]:
def read_json(json_data):
    for route in json_data['plan']['itineraries']:
        #Here we show the 3 different path
        print('-------- Route---------\n')
        #print(route)
        for step in route['legs']: 
            #Here we show all the step of the route   
            print('---Step---\n')
            #print(step)
            from_ = step['from']['name']
            to_ = step['to']['name']
            mode = step['mode']
            if('tripShortName' in step.keys()):
                route_id = step['tripShortName']
                #print(step)
            end_time = str(step['endTime'])
            duration = str(step['duration'])
            print('For the travel from {} to {} in {} \n'.format(from_, to_, mode))
            print('The duration is {} and the arrival time is {}\n'.format(time.strftime("%H:%M:%S", time.gmtime(float(duration))), time.strftime("%b %d %Y %H:%M:%S,%M", time.gmtime(float(end_time[:len(end_time)-3])))))
            if('tripShortName' in step.keys()):
                print('The route ID is {}'.format(route_id))
    

In [59]:
Days = [i for i in range(1, 32)]
Months = [i for i in range(1, 13)]
Hours = [i for i in range(0, 13)]
AM_PM = ['AM', 'PM']
Minutes = [0, 15, 30, 45]

In [82]:
StopName = Df_meta.select('StopName_Meta').distinct().collect()
StopName = [str(x).replace('"', "'") for x in StopName]
StopName = [str(x)[19:] for x in StopName]
StopName = [str(x).split("')")[0] for x in StopName]
StopName = sorted(StopName)

In [97]:
interact_manual(request, fromPlace=StopName, toPlace= StopName,Months = Months, Days= Days, Hours= Hours, AM_PM = AM_PM,Minutes=Minutes ,departure=True)

A Jupyter Widget

<function __main__.request>

### Test to see what we have in the data

It seems that we don't have data for bus and subway, at least near Lausanne 

After investigation it's seems that we have data for the LEB in Lausanne. 

In [198]:
df_tmp.printSchema()

root
 |-- Line_ID: string (nullable = true)
 |-- Line_ID_spec: string (nullable = true)
 |-- LineType: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- LineId: string (nullable = true)
 |-- LineType: string (nullable = true)
 |-- TripDate: string (nullable = true)
 |-- ArrivalTimeScheduled: string (nullable = true)
 |-- DepartureTimeScheduled: string (nullable = true)
 |-- StopName: string (nullable = true)



In [199]:
df_tmp.select('ProductId').distinct().show()

+---------+
|ProductId|
+---------+
|      BUS|
|     null|
|     Tram|
|      Zug|
|      Bus|
|   Schiff|
+---------+



In [221]:
df_tmp.filter(df_tmp['StopName'].rlike("Lausanne") & (col('ProductId') != 'Zug')).show(1)

+----------+------------+--------+---------+--------+----------+--------------------+----------------------+------------------+
|   Line_ID|Line_ID_spec|LineType|ProductId|LineType|  TripDate|ArrivalTimeScheduled|DepartureTimeScheduled|          StopName|
+----------+------------+--------+---------+--------+----------+--------------------+----------------------+------------------+
|43520-0171|        null|     435|      Bus|     435|14.03.2018|    14.03.2018 14:48|      14.03.2018 14:48|Lausanne, En Marin|
+----------+------------+--------+---------+--------+----------+--------------------+----------------------+------------------+
only showing top 1 row



In [217]:
df_tmp.where((col('ProductId') == 'Zug') & (col('Line_ID') == '108')).show(40)

+-------+------------+--------+---------+--------+----------+--------------------+----------------------+-------------------+
|Line_ID|Line_ID_spec|LineType|ProductId|LineType|  TripDate|ArrivalTimeScheduled|DepartureTimeScheduled|           StopName|
+-------+------------+--------+---------+--------+----------+--------------------+----------------------+-------------------+
|    108|         001|     ICE|      Zug|     ICE|13.09.2017|                null|      13.09.2017 11:13|          Basel SBB|
|    108|         001|     ICE|      Zug|     ICE|13.09.2017|    13.09.2017 11:19|                  null|       Basel Bad Bf|
|    108|         000|       R|      Zug|       R|13.09.2017|                null|      13.09.2017 07:41|             Morges|
|    108|         000|       R|      Zug|       R|13.09.2017|    13.09.2017 07:42|      13.09.2017 07:42|          La Gottaz|
|    108|         000|       R|      Zug|       R|13.09.2017|    13.09.2017 07:43|      13.09.2017 07:43|          Pré

In [207]:
df_tmp.where((col('ProductId') == 'Tram')).show(40)

+-------------+------------+--------+---------+-----------+--------+----------+--------------------+----------------------+--------------------+
|      Line_ID|Line_ID_spec|LineType|ProductId|     LineId|LineType|  TripDate|ArrivalTimeScheduled|DepartureTimeScheduled|            StopName|
+-------------+------------+--------+---------+-----------+--------+----------+--------------------+----------------------+--------------------+
|80250-02002-1|        null|       2|     Tram|85:3849:002|       2|13.09.2017|                null|      13.09.2017 06:01|Zürich,Kalkbreite...|
|80250-02002-1|        null|       2|     Tram|85:3849:002|       2|13.09.2017|    13.09.2017 06:02|      13.09.2017 06:02|   Zürich, Lochergut|
|80250-02002-1|        null|       2|     Tram|85:3849:002|       2|13.09.2017|    13.09.2017 06:03|      13.09.2017 06:03|Zürich, Zypressen...|
|80250-02002-1|        null|       2|     Tram|85:3849:002|       2|13.09.2017|    13.09.2017 06:04|      13.09.2017 06:04|Zürich,

In [197]:
df_tmp.where(df_tmp.Line_ID == '331').show(40)

+-------+------------+--------+---------+------+--------+----------+--------------------+----------------------+--------------------+
|Line_ID|Line_ID_spec|LineType|ProductId|LineId|LineType|  TripDate|ArrivalTimeScheduled|DepartureTimeScheduled|            StopName|
+-------+------------+--------+---------+------+--------+----------+--------------------+----------------------+--------------------+
|    331|         000|       R|      Zug|   331|       R|13.09.2017|                null|      13.09.2017 18:33| Les Ponts-de-Martel|
|    331|         000|       R|      Zug|   331|       R|13.09.2017|    13.09.2017 18:34|      13.09.2017 18:34|            Le Stand|
|    331|         000|       R|      Zug|   331|       R|13.09.2017|    13.09.2017 18:35|      13.09.2017 18:35|        Petit-Martel|
|    331|         000|       R|      Zug|   331|       R|13.09.2017|    13.09.2017 18:36|      13.09.2017 18:36|    Petit-Martel-Est|
|    331|         000|       R|      Zug|   331|       R|13.09